In [19]:
import pandas as pd
import string, re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import colorsys


In [8]:
# df = pd.read_excel("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_base.xlsx", engine='openpyxl')
# df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_allComment.csv")
# df_labelled = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})
import sys
sys.path.append('../')

from utils.schneider import countries_to_update, text_data_column, words_to_filter, replacements
from preprocessing.dataLoaders.schneider_data_loader import SchneiderDataLoader
from preprocessing.preprocessing import Preprocessor

df = pd.read_csv("../data/csv_files/schneider.csv")
schneiderDataLoader = SchneiderDataLoader(df, countries_to_update)
preprocessing = Preprocessor(
    schneiderDataLoader, 
    text_data_column,
    words_to_filter,
    replacements
)

df_preprocessed_full = preprocessing.preprocess()
# Filter rows based on 'non_empty_rows' column
df_preprocessed = df_preprocessed_full[df_preprocessed_full['non_empty_rows']]

/tmp/ipykernel_17459/1847981677.py:11: DtypeWarning: Columns (5,6,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/csv_files/schneider.csv")


In [7]:
df_preprocessed_full.columns

Index(['Account Country', 'Front Office Country', 'Clusters', 'Zone',
       'Operation', 'Primary Coverage Model', 'Account owner role',
       'Market Segment', 'Leading BU', 'Personas', 'State/Province',
       'Account ID Text', 'Classification Level 1 Name',
       'Classification Level 2', 'bFO Account ID', 'Job Title', 'Job Role',
       'Contact ID', 'bFO Contact ID', 'Survey ID', 'Unique ID',
       'Creation Date', 'Local Response Date', 'Response Date',
       'Accepted for reporting on date', 'Touchpoint', 'Interaction', 'Brand',
       'Survey language', 'Quarantine Rule Exception',
       'What is the Customer Feedback?', 'Likelihood to Recommend (SE)',
       'Customer_Comments', 'Translation_Customer_Comments',
       'Overall_Additional_Comments',
       'Translation_Overall_Additional_Comments', 'Overall Satisfaction',
       'Reason_for_Score_Comment', 'Translation_Reason_for_Score_Comment',
       'Anything_Else_Comment', 'Translation_Anything_Else_Comment', 'year',

In [13]:
########### Global Info 

nb_scores = df_preprocessed_full["Overall Satisfaction"].count()
nb_recommend_scores = df_preprocessed_full["Likelihood to Recommend (SE)"].count()
nb_comments = len(df_preprocessed)
pct_comment_by_score = nb_comments/nb_scores*100
pct_recommendScore_by_score = nb_recommend_scores/nb_scores*100

pct_recommendScore_by_score

34.60742300987798

In [15]:
# Count the frequency of each score
score_counts = df_preprocessed_full['Overall Satisfaction'].value_counts().sort_index()
recommend_score_counts = df_preprocessed_full['Likelihood to Recommend (SE)'].value_counts().sort_index()

# Create a bar chart
fig = go.Figure(data=go.Bar(x=score_counts.index, y=score_counts.values))
fig = go.Figure(data=go.Bar(x=recommend_score_counts.index, y=recommend_score_counts.values))
fig.update_layout(title='Distribution of Scores', xaxis_title='Score', yaxis_title='Frequency')
fig.show()

In [23]:
########### Descriptive Analysis

def create_donut_chart(df, label_column):
    # Count the frequency of each label
    label_counts = df[label_column].value_counts()

    # Create the donut chart
    fig = go.Figure(data=[go.Pie(labels=label_counts.index, 
                                 values=label_counts.values, 
                                 hole=.6,
                                #  marker=dict(colors=palette), 
                                # default:['#1f77b4',  '#ff7f0e',  '#2ca02c',  '#d62728',  '#9467bd',  '#8c564b',  '#e377c2',  '#7f7f7f',  '#bcbd22',  '#17becf']
                                 hovertemplate='%{label}: %{value}<extra></extra>'
                                )
                    ])
    
    fig.update_layout(
        legend_title_text=f"{label_column}",
        title={'text': f'Score Repartition by {label_column}',
               'y':0.9,'x':0.5,
               'xanchor': 'center','yanchor': 'top'},
        title_font=dict(size=20,
                        color='rgb(107, 107, 107)'),
        autosize=False,
        width=800,
        height=500
    )
    return fig

create_donut_chart(df_preprocessed_full, 'Zone')

In [24]:
# Create Sunburst Chart
def filter_docs(df, filter_column, filter_value):
    filter_mask = df[filter_column] == filter_value
    return df[filter_mask], filter_mask

def compute_labels_and_parents(data_df, levels):
    grouped = data_df.groupby(levels).size()

    # Initialize lists
    labels = []
    parents = []

    # Compute labels and parents lists
    for i in range(len(levels)):
        for label, value in grouped.items(): # loop over the rows
            if label[i] not in labels:
                labels.append(label[i]) 
                parents.append("") if label[i] in data_df[levels[0]].values else parents.append(label[i-1])
    
    return labels, parents

def prepare_dataframe(data_df, levels):
    df_tmp=data_df.copy()
    frequency_col_name = "Count"
    df_tmp[frequency_col_name] = df_tmp.groupby(levels[-1])[levels[-1]].transform('count')
    sunburst_df = df_tmp[levels + [frequency_col_name]].drop_duplicates()
    sunburst_df[frequency_col_name] = sunburst_df[frequency_col_name].fillna(0)
    
    return sunburst_df

def compute_values(sunburst_df, labels, levels):
    values = []
    processed_labels = set()  # This set will store the labels that have been processed
    for label in labels:
        if label not in processed_labels:  # Only process the label if it has not been processed before
            for level in levels:
                if label in sunburst_df[level].values:
                    value = sunburst_df.loc[sunburst_df[level] == label, 'Count'].sum()
                    values.append(value)
                    processed_labels.add(label)  # Add the label to the set of processed labels
                    break  # No need to check other levels for this label
                    
    return values

def get_level(label, data_df, levels, return_highest_level=True): 
    # Find the levels that the label appears in
    found_levels = [level for level in levels if label in data_df[level].values]
    if not found_levels:
        # raise ValueError(f"Label {label} not found")
        return "Unknown"
    # If return_highest is True, return the first level found (highest), else return the last level found (lowest)
    if return_highest_level:
        return found_levels[0]
    else:
        return found_levels[-1]

def compute_percentages(sunburst_df, data_df, labels, parents, values, levels):
    percentages = []
    percentages_class = []
    processed_labels = set()  # This set will store the labels that have been processed
    
    for level in levels:
        # Compute total count for each label in current level
        level_counts = sunburst_df.groupby(level)['Count'].sum()

        for i, label in enumerate(labels):
            if label not in processed_labels and label in sunburst_df[level].values:
                label_count = sunburst_df.loc[sunburst_df[level] == label, 'Count'].sum() 

                # Compute total count for the parent of current label with a condition if the label has a parent or not. 
                # Without parent, the total count is the count for all labels at this highest level
                total_count = values[labels.index(parents[i])] if parents[i]!='' else level_counts.sum()
                # Compute total count from original dataframe 
                total_count_class = data_df[get_level(label, data_df, levels)].value_counts().get(label)                

                # Compute percentage for current label
                percentage = (label_count / total_count) * 100
                percentages.append(percentage)
                percentage_class = (label_count / total_count_class) * 100
                percentages_class.append(percentage_class)
                processed_labels.add(label)  # Add the label to the set of processed labels

    return percentages, percentages_class

def transform_scale_list(lst,exponent=2, desire_sum=100):
    # Apply power transformation
    transformed_lst = [i**exponent for i in lst]

    # Scale the transformed data so that the total sum remains 100
    total = sum(transformed_lst)
    scaled_lst = [(i/total)*desire_sum for i in transformed_lst]
    return scaled_lst

def normalize_percentage_class(data_df, levels, labels, parents, percentages_class, scale_exponent=2):
    # Init
    normalized_percentages_class = [0]*len(percentages_class)
    normalized_percentages_class_transform = [0]*len(percentages_class)

    ##### Highest level

    # Compute normalized_percentage_class for labels at the highest level (without parent)
    highest_level_labels = [label for label in labels if get_level(label, data_df, levels) == levels[0]]
    highest_level_indices = [labels.index(label) for label in highest_level_labels]
    total_highest_level_percentage = sum(percentages_class[i] for i in highest_level_indices)
    for i in highest_level_indices:
        normalized_percentages_class[i] = percentages_class[i] / total_highest_level_percentage * 100

    # scale the values for the highest level
    normalized_percentages_class_transform = transform_scale_list(normalized_percentages_class,scale_exponent)

    ##### Other levels

    # Normalize percentages_class based on its parent
    for level in levels[1:]: #loop on the levels except the highest one
        level_labels = [label for label in labels if get_level(label, data_df, levels) == level]
        level_indices = [labels.index(label) for label in level_labels]

        # Loop to create the normalized_percentages_class list by level
        for indice in level_indices:
            sibling_indices = [j for j, parent in enumerate(parents) if parent == parents[indice]]
            total_sibling_percentage = sum(percentages_class[j] for j in sibling_indices)
            normalized_percentage_class = percentages_class[indice] / total_sibling_percentage * normalized_percentages_class[labels.index(parents[indice])]
            normalized_percentages_class[indice] = normalized_percentage_class

        # Loop to transform the normalized_percentages_class list by level : list normalized_percentages_class_transform
        indices_updated = [] # list to avoid several treatments on each label
        for indice in level_indices:
            if indice not in indices_updated:
                sibling_indices = [j for j, parent in enumerate(parents) if parent == parents[indice]]
                # Get & Transform norm_pct_class by family (brothers with the same parent)
                norm_pct_class_sibling = [normalized_percentages_class[i] for i in sibling_indices]
                norm_pct_class_sibling_transform = transform_scale_list(norm_pct_class_sibling, scale_exponent, desire_sum=normalized_percentages_class_transform[labels.index(parents[indice])])
                # Store the transform norm_pct_class in the normalized_percentages_class_transform list
                for i, sibling_ind in enumerate(sibling_indices):
                    normalized_percentages_class_transform[sibling_ind] = norm_pct_class_sibling_transform[i]
                    indices_updated.append(sibling_ind)

    return normalized_percentages_class_transform
    
def lighten_color(color, factor):
    # Check if the input is RGB or hexadecimal
    if isinstance(color, tuple):
        r, g, b = color
    else:
        # Remove the '#' from the start of the color code if it exists
        if color.startswith('#'):
            color = color[1:]
        r, g, b = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))

    # Convert RGB color to HLS
    h, l, s = colorsys.rgb_to_hls(r/255.0, g/255.0, b/255.0)
    # Increase the lightness
    l = max(min(l + factor, 1.0), 0.0)
    # Convert back to RGB
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    
    # Convert back to the original format
    if isinstance(color, tuple):
        return int(r*255), int(g*255), int(b*255)
    else:
        return '#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255))
    
def compute_color_list(data_df, labels, parents, levels, color_sequence):
    # Create a dictionary that maps each label to a color
    color_dict = {}

    # Sort your labels based on their level
    labels_sorted = sorted(labels, key=lambda label: get_level(label, data_df, levels), reverse=True)
    color_seq = color_sequence.copy()
    # Iterate over your reversed sorted labels list
    for i, label in enumerate(labels_sorted):
        parent = parents[labels.index(label)]  # Get the parent of the current label
        level = get_level(label, data_df, levels)  # Use your get_level function here

        if level == levels[0]:
            # Assign a unique color to each continent
            if label not in color_dict:
                # color_dict[label] = color_seq.pop(0)
                color_dict[label] = color_sequence[i % len(color_sequence)]
        else:
            # Assign a lighter shade of the parent's color to each country or city
            parent_color = color_dict[parent]
            color_dict[label] = lighten_color(parent_color, 0.1)
    return color_dict 

def add_unique_parent(labels, parents, values, unique_parent_name):
    unique_parent_value = sum(values[i] for i in range(len(parents)) if parents[i] == '')
    for i in range(len(labels)):
        if parents[i] == '':
            parents[i] = unique_parent_name
    labels.append(unique_parent_name)
    parents.append('')
    values.append(unique_parent_value)

    return labels, parents, values

def compute_lists(data_df, levels, color_sequence, class_column=None, class_value=None, unique_parent=False, unique_parent_name="Geo Levels"):
    if class_column and class_value==None:
        raise ValueError("When class_column is defined, class_value must be defined too and must be a value from the class_column column")
    
    work_df = data_df.copy()
    if class_column:
        work_df = filter_docs(data_df, class_column, class_value)[0]
    
    labels, parents = compute_labels_and_parents(work_df, levels)
    sunburst_df = prepare_dataframe(work_df, levels)
    values = compute_values(sunburst_df, labels, levels)
    # percentages, percentages_class, normalized_percentages_class = compute_percentages(sunburst_df, data_df, labels, parents, values, levels)
    percentages, percentages_class = compute_percentages(sunburst_df, data_df, labels, parents, values, levels)
    normalized_percentages_class_transform = normalize_percentage_class(data_df, levels, labels, parents, percentages_class, scale_exponent=1)
    color_dict = compute_color_list(data_df, labels, parents, levels, color_sequence)
    if unique_parent:
        labels, parents, values =  add_unique_parent(labels, parents, values, unique_parent_name=unique_parent_name)
        # Compute and add percentages for the unique_parent label
        percentages.append(100)
        percentages_class.append(len(filter_docs(data_df, class_column, class_value)[0])/len(data_df)*100) if class_column and class_value else percentages_class.append("None")
        # normalized_percentages_class.append(100) if class_column and class_value else normalized_percentages_class.append("None")
        normalized_percentages_class_transform.append(100) if class_column and class_value else normalized_percentages_class_transform.append("None")
        color_dict[unique_parent_name]='#ffffff'
    return labels, parents, values, percentages, percentages_class, normalized_percentages_class_transform, color_dict

def create_sunburst_fig(labels, parents, values, percentages, percentages_class, normalized_percentages_class_transform, color_dict, class_column=None, class_value=None, **sunburst_kwargs):
    # Create a list of hover texts that includes the percentage for each label
    hover_text = [f'{label}<br>Number of verbatims: {value}<br>Percentage: {percentage:.2f}%{f"<br>Percentage of {class_value}: {percentage_class:.2f}%" if class_column else ""}' for label, value, percentage, percentage_class in zip(labels, values, percentages, percentages_class)]

    fig = go.Figure(data=go.Sunburst(
        labels=labels,
        parents=parents,
        values=normalized_percentages_class_transform if class_column else values,
        branchvalues='total',
        hovertext=hover_text,
        hovertemplate='%{hovertext}<extra></extra>',  # Only use custom hover text and remove extra info            
        marker=dict(colors=[color_dict[label] for label in labels]),  # Set colors based on your mapping
        **sunburst_kwargs
    ))
    
    fig.update_layout(
        title={'text': f"Geographical Distribution",
               'y':0.95,'x':0.5,
               'xanchor': 'center','yanchor': 'top'},
        title_font=dict(size=20,
                        color='rgb(107, 107, 107)'),
        width=1100,
        height=1000
    )

    return fig

def sunburst(data_df, levels, color_sequence, unique_parent=True, class_column=None, class_value=None, **sunburst_kwargs):
    labels, parents, values, percentages, percentages_class, normalized_percentages_class_transform, color_dict = compute_lists(data_df, levels, color_sequence, unique_parent=unique_parent, class_column=class_column, class_value=class_value)
    fig = create_sunburst_fig(labels, parents, values, percentages, percentages_class, normalized_percentages_class_transform, color_dict, class_column=class_column, class_value=class_value, **sunburst_kwargs)
    return fig

levels = ['Zone','Clusters','Account Country'] # it has to be from the highest level to the lowest
color_sequence = ['#636EFA','#EF553B','#00CC96','#AB63FA','#FFA15A','#19D3F3','#FF6692','#B6E880','#FF97FF','#FECB52','#E763FA','#BA68C8','#FFA000','#F06292','#7986CB','#4DB6AC','#FF8A65','#A1887F','#90A4AE','#E53935','#8E24AA']
fig = sunburst(df_preprocessed_full, levels, color_sequence, unique_parent=False)#, class_column="single_emotion_label", class_value="disappointment")
fig

In [45]:
# Group by 'year' and calculate the total number of rows and the number of rows with comments
grouped = df_preprocessed_full.groupby('year').agg(
    total_rows=pd.NamedAgg(column='non_empty_rows', aggfunc='count'),
    rows_with_comments=pd.NamedAgg(column='non_empty_rows', aggfunc='sum')
)

# Calculate the percentage of rows with comments and without comments
grouped['percentage_with_comments'] = round(grouped['rows_with_comments'] / grouped['total_rows'] * 100,2)
grouped['percentage_without_comments'] = 100 - grouped['percentage_with_comments']

# Define a color sequence for the figure
color_sequence = ['#636EFA','#EF553B','#00CC96','#AB63FA','#FFA15A','#19D3F3','#FF6692','#B6E880','#FF97FF','#FECB52','#E763FA','#BA68C8','#FFA000','#F06292','#7986CB','#4DB6AC','#FF8A65','#A1887F','#90A4AE','#E53935','#8E24AA']

# Create a stacked bar chart using go.Figure
fig = go.Figure(data=[
    go.Bar(name='With Comments', x=grouped.index, y=grouped['percentage_with_comments'], marker_color=color_sequence, text=grouped['percentage_with_comments'].apply(lambda x: '{:.1f}%'.format(x)), textposition='auto', hovertemplate='Year: %{x}<br>Number of Comments: %{customdata}<extra></extra>', customdata=grouped['rows_with_comments']),
    go.Bar(name='Without Comments', x=grouped.index, y=grouped['percentage_without_comments'], marker_color="lightgrey", hovertemplate='Year: %{x}<br>Total Number of Rows: %{customdata}<extra></extra>', customdata=grouped['total_rows'])
])

# Change the bar mode
fig.update_layout(barmode='stack')

# Set chart title and labels
fig.update_layout(
    title_text='Percentage of Rows with Comments by Year',
    xaxis_title='Year',
    yaxis_title='Percentage of Rows',
    showlegend=False
)

# Show the figure
fig.show()